In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.sparse.linalg import svds
import seaborn as sns

In [2]:
# !unzip Data.zip

In [3]:
ratings=pd.read_csv('../Data/ratings.csv')
movies=pd.read_csv('../Data/movies.csv')
users=pd.read_csv('../Data/user.csv')

In [76]:
users=users.drop(users.columns[0],axis=1)
users.head()
movies=movies.drop(movies.columns[0],axis=1)

In [77]:
ratings

,index,user_id,movie_id,rating,timestamp
0,0,1,1193,5,978300760
1,1,1,661,3,978302109
2,2,1,914,3,978301968
3,3,1,3408,4,978300275
4,4,1,2355,5,978824291
...,...,...,...,...,...
1000204,1000204,6040,1091,1,956716541
1000205,1000205,6040,1094,5,956704887
1000206,1000206,6040,562,5,956704746
1000207,1000207,6040,1096,4,956715648


In [78]:
print("Number user:",len(np.unique(ratings['user_id'])))
print("Number movie:",len(np.unique(ratings['movie_id'])))

Number user: 6040
Number movie: 3706


Convert my ratings matrix to be one row per users and one column per movie. To do so, I'll pivot ratings to get that and call the new variable Ratings

In [79]:
Ratings=ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
Ratings.iloc[0,:]

movie_id
1       5.0
2       0.0
3       0.0
4       0.0
5       0.0
       ... 
3948    0.0
3949    0.0
3950    0.0
3951    0.0
3952    0.0
Name: 1, Length: 3706, dtype: float64

In [81]:
r=Ratings.values # convert to matrix
print(r.shape)
user_ratings_mean=np.mean(r,axis=1)
print(user_ratings_mean.shape)
rating_mean=r-user_ratings_mean.reshape(-1,1)

(6040, 3706)
(6040,)


In [82]:
# U,sigma,Vt=svds(rating_mean,k=50)
U,sigma,Vt=np.linalg.svd(rating_mean,full_matrices=False)
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(6040, 3706)
(3706,)
(3706, 3706)


In [83]:
sigma=np.diag(sigma)
print(sigma.shape)

(3706, 3706)


# Making predictions from the Decomposed Matrices

At this time, we need to make movie ratings predictions for all users. I can do it at once by following the math and matrix multiply $U$,$\Sigma$,$V^T$ back to get the rank K=50 to approximation of A

In [84]:
all_user_predicted_ratings=np.dot(np.dot(U,sigma),Vt)+user_ratings_mean.reshape(-1,1)

In [85]:
predict=pd.DataFrame(all_user_predicted_ratings,columns=Ratings.columns)
predict.head(3)

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,5.000000e+00,7.646661e-14,-7.104733e-14,-1.769418e-15,4.644202e-14,-2.517431e-14,-5.932754e-14,3.763656e-14,-2.544492e-14,1.838113e-14,3.809453e-15,-4.891226e-14,2.476491e-14,-3.137074e-14,6.421946e-14,4.111989e-14,5.035555e-14,-2.160078e-14,-1.137979e-14,7.630008e-14,-1.561251e-15,-7.528700e-15,-9.006684e-15,5.049433e-14,1.045067e-13,-5.162537e-15,1.748601e-14,-2.997602e-15,2.983030e-14,1.504352e-14,1.253164e-14,4.481138e-14,8.631984e-15,1.365574e-14,1.078304e-14,1.122713e-14,1.165734e-15,-2.359224e-16,2.893519e-15,6.869505e-16,...,-3.538836e-15,-3.032297e-15,3.518019e-15,1.106754e-14,2.699230e-15,4.649059e-16,4.163336e-16,6.453171e-16,1.776357e-15,-2.289835e-16,3.372302e-15,2.026157e-15,1.894318e-15,2.886580e-15,-2.914335e-16,3.809453e-15,5.537237e-15,8.500145e-15,-1.713907e-15,-2.289835e-15,-1.720846e-15,-7.840950e-16,1.533496e-15,2.088607e-15,6.133982e-15,-4.163336e-16,1.200429e-15,1.318390e-16,2.060851e-15,-1.283695e-15,-1.207368e-15,1.353084e-15,1.512679e-15,-6.730727e-16,-6.314393e-16,3.795575e-15,7.216450e-15,4.392320e-15,4.628242e-15,6.723788e-15
1,2.803313e-14,1.540434e-13,-2.748912e-13,8.368306e-14,2.148282e-14,-6.036838e-14,9.647838e-14,1.565414e-14,8.298917e-15,1.020017e-13,-1.357248e-14,1.318390e-14,6.705747e-14,8.096301e-14,1.013356e-13,-1.301736e-14,3.994027e-14,1.609823e-14,-6.106227e-15,2.772782e-14,1.000000e+00,3.108624e-15,3.566591e-14,-3.289036e-14,8.501533e-14,6.897261e-14,1.176836e-14,-1.448841e-14,5.551115e-14,3.194667e-14,-2.006728e-14,4.396483e-14,2.747802e-15,-6.045164e-14,3.719247e-15,-1.820766e-14,-8.298917e-15,-4.551914e-15,5.079270e-15,1.720846e-15,...,-4.163336e-15,7.494005e-16,3.413936e-15,1.093570e-14,8.187895e-15,3.247402e-15,-9.436896e-16,3.608225e-15,-9.436896e-16,4.718448e-16,3.774758e-15,-1.082467e-15,-3.164136e-15,6.411538e-15,7.910339e-15,6.356027e-15,-4.884981e-15,7.688294e-15,-4.357625e-15,8.548717e-15,-7.910339e-15,-6.106227e-16,-5.828671e-16,3.885781e-16,3.691492e-15,-4.413137e-15,-3.885781e-15,-5.606626e-15,-1.720846e-15,-2.803313e-15,1.221245e-15,-5.023759e-15,-4.968248e-15,-2.747802e-15,-1.831868e-15,8.798517e-15,1.243450e-14,2.525757e-15,-5.551115e-17,1.090794e-14
2,2.337019e-14,1.520312e-14,2.330774e-14,6.899342e-14,1.671580e-14,-4.281298e-15,3.393119e-14,-1.554312e-14,3.525652e-14,-2.846334e-14,-1.378064e-14,2.074729e-14,1.602191e-14,-3.281403e-14,-2.482042e-14,-3.005235e-14,-1.892236e-14,7.341350e-14,3.295975e-14,7.494005e-16,-1.654926e-14,5.953571e-15,-3.400058e-16,1.889461e-14,-9.950374e-15,7.355228e-16,1.050549e-14,8.111567e-15,-9.346690e-15,1.174061e-14,6.411538e-15,3.166911e-14,-2.485512e-14,1.128264e-14,3.066991e-15,-8.049117e-16,-4.045375e-15,4.891920e-15,3.601286e-15,-2.775558e-17,...,6.106227e-16,6.730727e-16,4.621303e-15,3.212708e-15,-2.470246e-15,-1.734723e-15,1.179612e-16,-7.632783e-16,1.908196e-15,1.929013e-15,2.081668e-17,2.449430e-15,4.975187e-15,-3.157197e-15,-2.872702e-15,-3.268219e-15,1.464107e-15,4.996004e-16,1.949829e-15,-2.900458e-15,7.216450e-16,-1.595946e-16,1.616762e-15,-2.171874e-15,-1.491862e-15,3.955170e-16,1.387779e-17,-9.714451e-17,6.314393e-16,3.747003e-16,2.983724e-15,2.636780e-16,1.207368e-15,2.310652e-15,-1.713907e-15,-1.665335e-16,-1.908196e-15,5.828671e-16,-1.332268e-15,-2.289835e-15


In [86]:
movie_ids=movies['movie_id'].astype('int')

In [87]:
def movie_recommend(userID,number_movie_recommendation):
    user_id=userID-1 #start from 0
    user_data_for_id=ratings[ratings.user_id==userID]
    already_movie_rated=user_data_for_id.merge(movies,how='inner',left_on='movie_id',right_on='movie_id').sort_values(['rating'])
    print("User ID {} has been voted {} number movies".format(userID,already_movie_rated.shape[0]))
    socres_predict=np.array(predict.iloc[user_id,:])
    c={movie_ids[i]:socres_predict[i] for i in range(len(socres_predict))}
    c=sorted(c.items(),key=lambda x:x[1],reverse=True)[1:number_movie_recommendation]
    indices=[i[0] for i in c]    
    return already_movie_rated,movies.iloc[indices]

In [88]:
already_rated,prediction=movie_recommend(1310,20)

User ID 1310 has been voted 24 number movies


In [89]:
already_rated.head(20)

,index,user_id,movie_id,rating,timestamp,title,genres
21,215949,1310,1231,2,974781963,"Right Stuff, The (1983)",Drama
2,215930,1310,1295,2,974782001,"Unbearable Lightness of Being, The (1988)",Drama
14,215942,1310,2313,2,974781839,"Elephant Man, The (1980)",Drama
22,215950,1310,1090,2,974781839,Platoon (1986),Drama|War
0,215928,1310,2988,3,974781935,Melvin and Howard (1980),Drama
19,215947,1310,1960,3,974782001,"Last Emperor, The (1987)",Drama|War
16,215944,1310,144,3,974781573,"Brothers McMullen, The (1995)",Comedy
20,215948,1310,2000,4,974781892,Lethal Weapon (1987),Action|Comedy|Crime|Drama
18,215946,1310,3526,4,974781892,Parenthood (1989),Comedy|Drama
17,215945,1310,3360,4,974781935,Hoosiers (1986),Drama


In [90]:
prediction

,movie_id,title,genres
3510,3579,I Dreamed of Africa (2000),Drama
1604,1650,Washington Square (1997),Drama
1219,1238,Local Hero (1983),Comedy
2127,2196,Knock Off (1998),Action
1122,1138,Dadetown (1995),Documentary
1038,1052,"Proprietor, The (1996)",Drama
1889,1958,Terms of Endearment (1983),Comedy|Drama
1225,1245,Miller's Crossing (1990),Drama
2954,3023,My Best Girl (1927),Comedy|Romance
3200,3269,Forever Young (1992),Adventure|Romance|Sci-Fi


In [91]:
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate

In [92]:
reader=Reader()
data=Dataset.load_from_df(ratings[['user_id','movie_id','rating']],reader)
svd=SVD()
cross_validate(svd,data,measures=['RMSE'],cv=5)

{'fit_time': (39.12251043319702,
  39.3544979095459,
  39.383366107940674,
  39.450085401535034,
  39.450063943862915),
 'test_rmse': array([0.87424291, 0.87457534, 0.87178999, 0.8734929 , 0.87379081]),
 'test_time': (2.4240834712982178,
  2.366638422012329,
  2.415132999420166,
  2.320676803588867,
  2.381477117538452)}